In [1]:
# %% Import Required Libraries

import pandas as pd
import numpy as np

In [2]:
# %% Data Import

file_path = 'rundata.xlsx'
df = pd.read_excel(file_path, sheet_name='Rundata_Sheet')

In [ ]:
# %% Initial Inspection of Data
print(df.iloc[0])

In [ ]:
# %% Initial Inspection of Data
df.info()

In [5]:
# %% Removing Non-Essential Columns

In [ ]:
# Remove 'Bildlänk' column
# Info.

n = df.shape[0]
r = df['Bildlänk'].isnull().sum()
p = (r*100)//n

print(f'Null values in the "Bildlänk" column = [{r}] that counts for [{p}%].')

# Drop 'Bildlänk' column

df = df.drop(columns=['Bildlänk'], axis=1)

print("The column 'Bildlänk' has been dropped as it includes non-essential information.")

In [ ]:
# Remove 'Referens' column
# Info.

n = df.shape[0]
r = df['Referens'].isnull().sum()
p = (r*100)//n

print(f'Null values in the "Referens" column = [{r}] that counts for [{p}%].')

# Drop 'Referens' column

df = df.drop(columns=['Referens'], axis=1)

print("The column 'Referens' has been dropped as it includes non-essential information.")

In [ ]:
# Remove 'Alternativt signum' column
# info.
n = df.shape[0]
r = df['Alternativt signum'].isnull().sum()
p = (r*100)//n

print(f'Null values in the "Alternativt signum" column = [{r}] that counts for [{p}%].')

# drop 'Alternativt signum' column

df = df.drop(columns=['Alternativt signum'], axis=1)

print("The column 'Alternativt signum' has been dropped as it includes non-essential information.")

In [ ]:
# Remove 'Övrigt' column
# info.

n = df.shape[0]
r = df['Övrigt'].isnull().sum()
p = (r*100)//n

print(f'Null values in the "Övrigt" column = [{r}] that counts for [{p}%].')

# Drop'Övrigt' column

df = df.drop(columns=['Övrigt'], axis=1)

print("The column 'Övrigt' has been dropped as it includes non-essential information.")

In [ ]:
# %% Renaming Columns

column_names = {
    'Signum':'sigma',
    'Plats': 'plats',
    'Socken':'socken',
    'Härad': 'harad',
    'Kommun': 'kommun',
    'Placering': 'placering',
    'Koordinater': 'koordinater',
    'Urspr. plats?': 'ursprunglig_plats',
    'Nuv. koord.': 'nuvarande_koordinater',
    'Sockenkod/Fornlämningsnr.': 'fornlamningsnummer',
    'Runtyper': 'runtyp',
    'Korsform': 'korsform',
    'Period/Datering': 'datering',
    'Stilgruppering':'stilgrupp',
    'Ristare': 'ristare',
    'Materialtyp':'materialtyp',
    'Material':'material',
    'Föremål': 'foremal'
}

df.rename(columns=column_names, inplace=True)
print('New Dataset column names:\n')

# Info.
for old, new in column_names.items():
    print(f'[{old}]: {new}')

In [11]:
# %% Cleaning Categorical Columns

# Replacing entries with only a question mark as value [?] to be able to replace it with NaN later. 
# df['harad'] = df['harad'].str.replace('?', '') can not be used as '' is not NaN.
# We use regex (Regular Expression): ^ regular expression anchor matches beginning of a string. 
# $ regex anchor matches the end of a string.
# \? we used backslash to escape ? as it is a special regex character.
# Without regex=True is r"^\?$" exactly ^\?$ and nothing else.

"""
    (method) def replace(to_replace: _str | list | dict | Series | float | None = ...,
    value: Scalar | NAType | dict | list | None = ...,
    *,
    inplace: Literal[False] = ...,
    limit: int | None = ...,
    regex: _bool = ...,
    method: ReplaceMethod = ...
    ) -> Series
"""

categorical_list = ['plats','socken', 'harad', 'kommun']

for col in categorical_list:
    df[col] = df[col].replace(to_replace= r'^\?$', value= np.nan, regex=True)


In [12]:
# Replace ? substring from entries in 'harad' column
# Note: The first .replace() method works on any data type, while the second .replace() is specifically for string data.
# Each method has it's own parameters

"""
    (method) def replace(
    pat: str,
    repl: str | ((Match) -> str),
    n: int = ...,
    case: bool | None = ...,
    flags: int = ...,
    regex: bool = ...
    ) -> Series
"""

df['harad'] = df['harad'].str.replace(pat='?,', repl='')

In [ ]:
# Replace NaN values with 'missing' in categorical columns

categorical_columns = [
    'sigma', 'plats','socken', 'harad', 'kommun', 'placering',
    'ursprunglig_plats', 'runtyp', 'datering','stilgrupp', 'ristare',
    'materialtyp','material', 'foremal'
]

print("Number of empty values (NaN) replaced with 'missing' in the dataset columns:")
for col in categorical_columns:
    
    nan_count = df[col].isnull().sum()
    print(f'[{col}]: [{nan_count}]')
    
    df[col] = df[col].fillna(value='missing')

In [14]:
# %% Cleaning 'foremal' Column

df['cleaned_foremal'] = df['foremal'].str.lower()
df['cleaned_foremal'] = df['cleaned_foremal'].str.replace(pat=r'[0-9]', repl='', regex=True)

marks_list = ['(?)', '[]', '()', '?']

for item in marks_list:
    df['cleaned_foremal'] = df['cleaned_foremal'].str.replace(pat=item, repl='')

df['cleaned_foremal'] = df['cleaned_foremal'].str.strip()

foremal_list = [
    'fragment av runsten', 'fragment av gravhäll', 'läderslida', 'laggskålsbotten',
    'putsinskrift', 'dopfunt', 'runben, revben', 'runmålning','stavkyrkegrafitti', 'trästycke'
]

for item in foremal_list:
    df.loc[df['cleaned_foremal'].str.contains(pat=item, case=False, na=False), 'cleaned_foremal'] = item

df.loc[df['cleaned_foremal'].str.contains(pat='runben, revben, fragment', case=False, na=False), 'cleaned_foremal'] = 'runben, revben'
df.loc[df['cleaned_foremal'].str.contains(pat='revben, fragment, st', case=False, na=False), 'cleaned_foremal'] ='revben, fragment'


In [ ]:
# %% Removing Rows with 'okänd' in 'materialtyp'

okänd_count_materialtyp = (df['materialtyp'] == 'okänd').sum()
print(f"Number of rows to be dropped with 'okänd' entry in 'materialtyp': [{okänd_count_materialtyp}]")

df = df.drop(index=df[df['materialtyp'] == 'okänd'].index) # OR df.drop(index=df[df['materialtyp'] == 'okänd'].index, inplace=True)


In [ ]:
# %% Removing Rows with 'missing' in both 'materialtyp' and 'datering'

missing_count_materilatype = (df['materialtyp'] =='missing').sum()
missing_count_datering = (df['datering'] =='missing').sum()
print(f"Number of rows with 'missing' in 'materialtyp': [{missing_count_materilatype}] and in 'datering': [{missing_count_datering}]")

df = df.drop(index=df[(df['materialtyp'] =='missing') & (df['datering'] =='missing')].index) # OR df.drop(index=df[(df['materialtyp'] =='missing') & (df['datering'] =='missing')].index, inplace=True)


In [17]:
# %% Cleaning 'material' Column

df['cleaned_material'] = df['material'].str.lower()
df['cleaned_material'] = df['cleaned_material'].str.replace(pat='(?)', repl='')
df['cleaned_material'] = df['cleaned_material'].str.replace(pat='?', repl='')
df['cleaned_material'] = df['cleaned_material'].str.strip()

material_list = ['brons', 'guld','silver', 'kalkputs', 'trä', 'amfibolit', 'ben', 'bok', 'ek', 'furu']

for item in material_list:
    df.loc[df['cleaned_material'].str.contains(pat=item, case=False, na=False), 'cleaned_material'] = item


In [ ]:
# %% Cleaning 'korsform' Column

n = df['korsform'].str.contains(pat=r"\[Klassificerad av redaktionen\]").sum()
print(f'The "korsform" column include [{n}] entries with [Klassificerad av redaktionen] substring.')

df['korsform'] = df['korsform'].str.replace(pat='[Klassificerad av redaktionen]', repl='')

print('Substring [Klassificerad av redaktionen] has been removed from the entries of the "korsform" column.')
print(f'After removal it includes [{df['korsform'].str.contains(pat=r"\[Klassificerad av redaktionen\]").sum()}] entries.')


In [19]:
# %% Cleaning 'stilgrupp' Column

df['cleaned_stilgrupp'] = df['stilgrupp']

stilgrupp_list = [
    '[Gräslund muntligt]', '[Klassificerad av redaktionen]', '[Källström 2007a:362]',
    '[Ljung muntligt]', '[NOR 2004:27]', '[Fv 2009]', '[Fv 2012]', '[Källström i SRI 10 Dig. suppl.]', '[En av, Ljung muntligt]',
    ' [Källström 2007a:361]', '[Källström 2007a:407]'
]

for item in stilgrupp_list:
    df['cleaned_stilgrupp'] = df['cleaned_stilgrupp'].str.replace(pat=item, repl='')

df['cleaned_stilgrupp'] = df['cleaned_stilgrupp'].str.replace(pat='sid a: Pr2, sid b: Pr2 - Pr3', repl='missing')
df['cleaned_stilgrupp'] = df['cleaned_stilgrupp'].replace(to_replace=r'^\?$', value=np.nan, regex=True)
df['cleaned_stilgrupp'] = df['cleaned_stilgrupp'].str.replace(pat='?', repl='')
df['cleaned_stilgrupp'] = df['cleaned_stilgrupp'].str.strip()
df['cleaned_stilgrupp'] = df['cleaned_stilgrupp'].fillna(value='missing')


In [ ]:
# %% Cleaning 'datering' Column

df['datering'] = df['datering'].str.strip().str.lower()

df['datering'].info()

list_unique_dates = list(df['datering'].unique())
list_unique_dates.sort()
list_unique_dates


In [22]:
# %% Categorizing Dates
# Date Lists

### 'Folkvandringstid 160–375'
early_migration_period = [
    'u 160-210/220 (imer 2007)',
    'u 160-250/260 (imer 2007)',
    'u 160-310/320 (imer 2007)',
    'u 160-375/400 (imer 2007)',
    'u 210/220-250/260',
    'u 210/220-250/260 (imer 2007)',
    'u 210/220-310/320 (imer 2007)',
    'u 210/220-375 (imer 2007)',
    'u 160-520/530 (imer 2007)',
    'u 160-560/570 (imer 2007)',
    'u 210/220-375/400',
    'u 250/260-310/320 (imer 2007)',
    'u 310/320-375/400 (imer 2007)',
    'u ca 160 (imer 2007)',
    'u ca 200-400'
]

### 'Sent folkvandringstid 375–500'
late_migration_to_early_vendel = [
    '§a=u 375/400-560/570 (imer 2007), §b=v',
    'u 375/400-460/470 (imer 2007)',
    'u 375/400-520/530 (imer 2007)',
    'u 375/400-560/570 (imer 2007)',
    'u 400-450',
    'u 400-500',
    'u 400-500-t',
    'u 450-475 (imer 2007)',
    'u 450-500 (imer 2007)',
    'u 460/470-520/530 (imer 2007)',
    'u 460/470-560/570 (imer 2007)',
    'u 475-525 (imer 2007)',
    'u 350-400 (knirk 2011)',
    'u 350/375-400 (imer 2007)',
    'u 375/400-675 (imer 2007)',
    'u 500-t?',
    'u ej äldre än m 500-t',
    'u 520/530-560/570 (imer 2007) ; u 550-700 (fv 2011)',
    'u 520/530-560/570? (imer 2007)',
    'u ca 450-550',
    'u ca 500',
    'u s 400-t - b 500-t',
    'u 400-t'
]

### 'Tidig vendeltid 500–600'
early_vendel_period = [
    'u 500-t',
    'u 500-t (thuesen)',
    'u 520/530-560/570 (imer 2007)',
    'u 520/530-700 (imer 2007)',
    'u 525-560/570 (imer 2007)',
    'u 560/570-600 (imer 2007)',
    'u ca 650 - ca 700'
]

### 'Mellanvendeltid 600–700'
mid_vendel_period = [
    'u 650-700 (grønvik)',
    'u 560/570-700 (imer 2007)',
    'u 600-700 (imer 2007)',
    'u 630-675/700 (imer 2007)'
]

### 'Sent vendeltid 700–800'
late_vendel_to_viking = [
    'u 560/570-775/800 (imer 2007)',
    'u 675-725 (imer 2007)',
    'u ca 650 - ca 700'
    'u/v 600-800',
    'u/v 675-775/800 (imer 2007)',
    'u/v 600-800'
]

### 'Oklar/allmän vendeltid'
unclear_vendel_period = [
    'u',
    'u 400-650',
    'u 400-t, något yngre än g 204',
    'u ej äldre än m 500-t'
    'u eller sentida?',
    'u förvikingatida',
    'u/v 560/570-775/800 (imer 2007)',
    'u? (oviss datering)',
    'u? föremålet är från 600-t, inskriften kan vara urnordisk eller övergångsform',
    'u? osäker datering',
    'u eller sentida?'
]

### 'Tidig medeltid 1000–1150'
early_medieval = [
    'm 1050-1100',
    'm 1050-1150',
    'm ca 1065-1080',
    'm ca 1100',
    'm 1100-t',
    'm 1100-1150',
    'm 1100-1150 (arkeologisk datering)',
    'm 1100-1150 (dendrokronologi)',
    'm 1115-1130',
    'm 1125-1175',
    'm 1130-1150',
    'm 1140-1175',
    'm 1150-1175',
    'm 1150-1200',
    'm 1160-1190',
    'm s 1000-t',
    'm s 1000-t - b 1100-t',
    'm s 1000-t - s 1100-t',
    'm tidigast ca 1400'
]

### 'Högmedeltid 1150–1300'
high_medieval = [
    'm 1150-1225',
    'm 1175-1225',
    'm 1180-1190',
    'm 1200-1250',
    'm 1200-1300',
    'm 1250-1300',
    'm 1275-1300',
    'm s 1200-t',
    'm m 1200-t',
    'm s 1200-t - 1300-t',
    'm s 1200-t - b 1300-t',
    'm (1200-1300-t)',
    'm (1350-1400, konsthistorisk datering)',
    'm §a-§e: 1280-1296, §f: 1300-t',
    'm 1050-1361, troligen 1300-t', 
    'm 1100 - b 1200-t', 
    'm 1100/1200-1400-t', 
    'm 1100 - b 1200-t',
    'm 1100-1175',
    'm 1100-1200',
    'm 1100-1200-t',
    'm 1100-1200-t (stratigr.)',
    'm 1100-1250',
    'm 1100-1250, trol. ca 1200',
    'm 1100-1280',
    'm 1100-1300',
    'm 1150-1250',
    'm 1150-1350',
    'm 1175-1200',
    'm 1175-1275',
    'm 1175-1300',
    'm 1175-1325',
    'm 1175-1350',
    'm 1180-1250',
    'm 1184',
    'm 1190-t',
    'm 1200-1230',
    'm 1200-1350 (stratigr.)',
    'm 1200-t',
    'm 1200-t (stratigr.)',
    'm 1200-t - 1300-t',
    'm 1200-t - b 1300-t',
    'm 1200-t el. b 1300-t',
    'm 1200-t eller ngt äldre',
    'm 1220-1235',
    'm 1225-1275',
    'm 1225-1325',
    'm 1228',
    'm 1230-1260',
    'm 1238',
    'm 1250-1275',
    'm 1250-1325',
    'm 1250-1350',
    'm ca 1150',
    'm ca 1150 eller senare',
    'm ca 1160-1170',
    'm ca 1170',
    'm ca 1175',
    'm ca 1175-1200',
    'm ca 1180',
    'm ca 1185',
    'm ca 1198',
    'm ca 1200',
    'm ca 1200 (stratigr.)',
    'm ca 1200 eller lite senare',
    'm ca 1200-1225',
    'm ca 1200-1250',
    'm efter 1170',
    'm efter 1198',
    'm efter 1200',
    'm efter ca 1200',
    'm ej senare än 1250',
    'm före 1170',
    'm före 1198',
    'm 1100-t (stratigr.)',
    'm 1100-t - 1200-t',
    'm 1100-t - b 1200-t',
    'm 1100-t?',
    'm 1100-talets mitt',
    'm 1150-1200 (stratigr.)',
    'm 1175-1225/1250',
    'm 1200-1250 (stratig.)',
    'm 1200-1250 (stratigr.)',
    'm 1200-1300, trol. efter 1250',
    'm 1200-1300-t',
    'm 1200-1300-t (stratigr.)',
    'm 1200-1400',
    'm 1200-1400-t',
    'm 1200-1500 (a), efter m 1200-t (b)',
    'm 1200-t(?)',
    'm 1200-t?',
    'm 1200-t? (stratigr.)',
    'm 1200-talets mitt eller något tidigare (stratigr.) (fv 1972:98)',
    'm 1250-1300 (?)',
    'm 1250-1300 (arkeologisk datering)',
    'm 1250-1300 (stratigr.)',
    'm 1250-1400',
    'm 1250-1425',
    'm 1263(?)',
    'm 21 juni 1197 (?)',
    'm andra hälften av 1100-t',
    'm andra hälften av 1100-t(?)',
    'm andra hälften av 1100-t.',
    'm andra hälften av 1200-t',
    'm andra hälften av 1200-t (stratigr.) (fv 1972:261)',
    'm b 1100-t',
    'm b 1200-t',
    'm b 1200-t (stratigr.)',
    'm b 1200-t - 1250',
    'm b 1300-t',
    'm början av 1100-t',
    'm början av 1200-t',
    'm ca 1100-1150',
    'm ca 1100?',
    'm ca 1170 (?)',
    'm ca 1200 (?)',
    'm ca 1200-1300',
    'm ca 1230-1260',
    'm ca 1240',
    'm ca 1248',
    'm ca 1248 ?',
    'm ca 1250',
    'm ca 1250 (?)',
    'm ca 1250 (arkeologisk datering)',
    'm ca 1250-1300',
    'm ca 1250-1350',
    'm ca 1275',
    'm ca 1300 (stratigr.) (fv 1972:262)',
    'm efter 1248',
    'm efter 1250',
    'm efter 1283 (seim 2010)',
    'm efter ca 1200, trol. 1400-t (a)',
    'm eller sentida',
    'm eller sentida?',
    'm ev. renässans',
    'm eventuellt sentida',
    'm före 1413 ?',
    'm första hälften av 1100-t',
    'm första hälften av 1200-t',
    'm juni 1197 (?)',
    'm m 1100-t',
    'm m 1200-t (dendrokr.)',
    'm m 1200-t - 1300',
    'm mitten av 1100-t',
    'm mitten av 1200-talet (stratigr.) (fv 1972:103)',
    'm s 1000-t - 1100-t',
    'm s 1100-t',
    'm s 1100-t - 1200-t',
    'm s 1100-t - 1361',
    'm s 1100-t - b 1200-t',
    'm s 1200-t (arkeologisk datering)',
    'm s 1200-t (konsthistorisk datering)',
    'm s 1200-t - omkr. 1300',
    'm senmedeltid?',
    'm slutet av 1100-t',
    'm slutet av 1100-t (stratigr.) (svärdström 1982:32)',
    'm slutet av 1200-t?',
    'm slutet av 1300-t',
    'm tidigt 1200-t',
    'm trol. första hälften av 1200-t',
    'm trol. senare hälften av 1200-t',
    'm? ca 900-1200',
    'm trol. 1100-t',
    'm troligen 1100-t',
    'm strax efter 1248',
    'm 1100-1200-t (stratigr.), 1200-1300-t (runogr.)'
]

### 'Senmedeltid 1300–1500'
late_medieval = [
    'm 1300-1400',
    'm 1350-1400',
    'm 1400-t',
    'm 1450-1550',
    'm senmedeltid',
    'm 1400-t - 1500-t',
    'm s 1300-t - 1400',
    'm s 1300-t - ca 1400',
    'm ca 1330',
    'm 1375-1400',
    'm 1100-1350',
    'm 1100-1400',
    'm 1100-1500',
    'm 1100/1200-1400-t',
    'm 1270-1315',
    'm 1275-1325',
    'm 1280-t',
    'm 1286 (dendrokronologisk datering)',
    'm 1294',
    'm 1298-99 (dendrokronologisk datering)',
    'm 1300-1350',
    'm 1300-1375',
    'm 1300-t',
    'm 1300-t (stratigr.)',
    'm 1300-t - 1400-t',
    'm 1305-1325',
    'm 1311 (1310)',
    'm 1317',
    'm 1324',
    'm 1326',
    'm 1328-33 (dendrokronologisk datering)',
    'm 1332-1413',
    'm 1335',
    'm 1350',
    'm 1350-1450 (stratigr.)',
    'm 1375-1450',
    'm 1383 eller strax efter',
    'm 1400-1425',
    'm 1400-1450',
    'm 1419',
    'm efter m 1200-t',
    'm efter m 1200-t - b 1300-t',
    'm ej senare än ca 1400',
    'm före 1248',
    'm före 1332',
    'm före 1347',
    'm före 1393',
    'm före 1413',
    'm före 1476',
    'm före omkr. 1400',
    'm ca 1300',
    'm ca 1300 - m 1300-t',
    'm ca 1300, samtidig med n 297',
    'm ca 1300/1300-1330',
    'm ca 1332',
    'm ca 1335',
    'm ca 1350',
    'm ca 1350 (arkeologisk datering)',
    'm ca 1350-1400',
    'm ca 1413',
    'm ca 1429',
    'm senromansk',
    'm senast m 1300-t',
    'm efter m 1300-t',
    'm 1479',
    'm 1487 27 oktober',
    'm 1492',
    'm 1493',
    'm 1495 (dendrokronologisk datering)',
    'm 1475-1500',
    'm 1300-1400-t',
    'm 1300-t?',
    'm 1350-1400 (arkeologisk datering)',
    'm 1361?',
    'm 1400-t (a)',
    'm 1400-t (c)',
    'm 1400-t (ev. äldre) (a)',
    'm 1400-t (möjl. yngre)',
    'm 1400-t (senast 1445)',
    'm 1400-t - 1500-t (a)',
    'm 1400-t - ca 1500',
    'm 1400-t möjl. äldre',
    'm 1400-t möjl. äldre (a)',
    'm 1400-t möjl. äldre (d) el. yngre (f)',
    'm 1400-t(?) (b)',
    'm 1400-t, korstolen är från 1300-t',
    'm 1400-t?',
    'm 1445',
    'm 1449',
    'm 1459',
    'm 1500-t?',
    'm 1514 23 november',
    'm 1553 5 augusti',
    'm 1582(?) (a)',
    'm 16 juli 1345',
    'm b 1400-t',
    'm andra hälften av 1300-t',
    'm andra hälften av 1300-t - omkr. 1400',
    'm ca 1400 (?)',
    'm ca 1479',
    'm ca 1500?',
    'm ev. 1349',
    'm efter 1332',
    'm efter 1347',
    'm första hälften av 1300-t',
    'm s 1400-t - ev. 1500-t',
    'm sista hälften av 1300-t',
    'm trol. 1500-t (d)',
    'm? ca 1500',
    'm m 1300-t',
    'm m 1300-t - 1400',
    'm m 1300-t - 1400-t',
    'm s 1300-t',
    'm s 1300-t - omkr. 1400',
    'm s 1400-t - 1500-t',
    'm trol. 1300-t',
    'm trol. 1300-t.',
    'm trol. 1400-t',
    'm trol. 1400-t (a)',
    'm trol. b 1300-t',
    'm troligen 1400-t',
    'm omkr. 1300'
]

### 'Osäker/allmän medeltida'
general_medieval = [
    'm?',
    'm går ej att tidfästa närmare',
    'm, funnet i ett 1300-talslager',
    'm äldre än 1198',
    'm troligen',
    'm högmedeltid',
    'm romansk',
    'm hög-senmedeltid',
    'm hög/senmedeltid',
    'm övergångstid',
    'm? (eventuellt v)',
    'm? andra hälften av 1000-t',
    'm? ej äldre än 1500-t',
    'm? eller från nyare tid?',
    'm? eventuellt eftermedeltida',
    'm? kan vara sentida',
    'm? sen medeltid, eller ännu yngre',
]

### 'Modern tid 1500–1700'
early_modern_period = [
    'sentida 1500-1800',
    'sentida 1500-t',
    'sentida 1525',
    'sentida 1527',
    'sentida 1568, 1621',
    'sentida 1600-t',
    'sentida ca 1550',
    'sentida s 1500-t - b 1600-t',
    'm 1500-t (agertz 2002)',
    'm trol. 1500-t',
    'm ca 1500',
    'm trol. s 1500-t',
    'm 1500-t (möjl. 1600-t)',
    'm 1506 4 april',
    'm 1508'
]

### 'Modern tid 1700–1800'
eighteenth_century = [
    'sentida 1700-t',
    'sentida 1700-t(?)',
    '1700-talet 1700–1800'
]

### 'Modern tid 1800–1900'
nineteenth_twentieth_century = [
    'sentida (1900-tal)',
    '1800–1900-talet',
    '1700-talet 1700–1800',
    '1800–1900-talet'
]

### 'Allmän/osäker sentida'
general_sentida = [
    'sentida',
    'troligen sentida', 
    'sentida?'
]

### 'Tidig vikingatiden 700-800'
early_viking_age = [
    'v 700-750 (dk sk 5)',
    'v 700-750 (imer 2007)',
    'v 700-t',
    'v 775-810',
    'v ca 800',
    'v ca 800 (arkeologisk datering)',
    'v tidigvikingatida',
    'v första hälften av 800-t',
    'v helnæs-gørlev, ca 750-900',
    'v helnæs-gørlev/för-jelling',
    'v 675-900 (imer 2007)',
    'v 700-800-t',
    'v 700-900',
    'v 775-875 (imer 2007)',
    'v 775/800-900 (imer 2007)',
    'v 800-1000',
    'v 800-1000 (imer 2007)',
    'v 800-1100',
    'v 800-900 (imer 2007)',
    'v 800-900-t',
    'v 800-t',
    'v 800-t (§e 1000-t)',
    'v 800-t - 900-t',
    'v 825-850 (imer 2007)',
    'v 850-925 (imer 2007)',
    'v 850-950 (imer 2007)',
    'v 875-925 (imer 2007)',
    'v s 700-t - b 900-t',
    'v ev. 800-t',
    'v m 800-t - s 800-t'
]

### 'Vikingatiden 800-1050'
viking_age = [
    'v m 800-t',
    'v m 800-t - s 800-t'
    'v ev. 800-t',
    'v ca 800-850',
    'v (1000-1100)',
    'v (950-1000, typologisk datering)',
    'v (efter 1025, dendrokronologisk datering)',
    'v ca 900',
    'v ca 940',
    'v ca 950',
    'v ca 1100',
    'v 1000-1050',
    'v 1000-1050 (imer 2007)',
    'v 1000-1100',
    'v 1000-1100 (imer 2007)',
    'v 1000-1150',
    'v 1000-t',
    'v 1000-t (arkeologisk datering)',
    'v 1000-t - b 1100-t',
    'v 1010-1020-t',
    'v 1020-1050',
    'v 1025-1050',
    'v 1030-1050 (arkeologisk datering)',
    'v 1030-t',
    'v 1040-t',
    'v 1050-1100',
    'v 1050-1100 (arkeologisk datering)',
    'v 1050-1100 (stratigr.)',
    'v 1050-1150',
    'v 1060-t (numismatisk datering)',
    'v 1065-1075',
    'v 1065-1085',
    'v 1067-1093',
    'v 1070-1090',
    'v 1073-1093',
    'v 1075-1085',
    'v 1075-1095',
    'v 1075-1100',
    'v 1100-1125',
    'v efter-jelling',
    'v jelling, 2. hälften av 900-t',
    'v jelling, efter år 934',
    'v terminus ante quem: 1039/1046',
    'v efter 1050',
    'v ca 1025',
    'v ca 1025-1050',
    'v ca 1040',
    'v ca 1065 (numismatisk datering)',
    'v ca 1075',
    'v 900-1000 (imer 2007)',
    'v 900-1000-t',
    'v 900-1050 (imer 2007)',
    'v 900-950 (imer 2007)',
    'v 900-t',
    'v 900-t - 1000-t',
    'v 900-t el. b 1000-t',
    'v 900-t?',
    'v 930-950',
    'v 950-1000',
    'v 950-1000 (imer 2007)',
    'v 950-1050',
    'v 950-1100 (imer 2007)',
    'v 950-975',
    'v 975-1100',
    'v 980-990',
    'v 990-1000 (imer 2007)',
    'v 990-1003 (myntet) 1000-1050 (runinskriften) (imer 2007)',
    'v 990-t',
    'v 993-1013',
    'v andra hälften av 1000-t',
    'v andra hälften av 900-t',
    'v b 1000-t',
    'v b 1000-t (oornerad)',
    'v b 1000-t?',
    'v b 1070-t',
    'v b 1100-t',
    'v b 800-t',
    'v b 900-t',
    'v ca 1000',
    'v ca 1000-1030',
    'v ca 1000-1050',
    'v (s 900-t - ca 1000)',
    'v 1000-t?',
    'v s 800-t',
    'v s 900 - m 1000-t',
    'v s 900-t',
    'v s 900-t - 1100',
    'v s 900-t - m 1000-t',
    'v s 900-t - m 1100-t',
    'v senare delen av 900-t - omkr. 1000',
    'v senast m 1100-t',
    'v tidig oornamenterad',
    'v trol. 1000-1050',
    'v trol. 1000-t',
    'v trol. ca 1000',
    'v ung, ev 1100-t',
    'v m 1000-t',
    'v omkr. 1000',
    'v s 1000-t',
    'v s 1000-t - b 1100-t',
    'v s 1000-t el. b 1100-t',
    'v före 1000',
    'v första delen av 1000-t',
    'v första hälften av 1000-t',
    'v utom d som är m troligen 1100-t',
    'v? (1000-1100-t, arkeologisk datering)',
    'v? 1000-t',
    'v? 1000-t (arkeologisk datering)',
    'v? 1050-1100',
    'v? 1050-1100?',
    'v ej senare än ca 1100'
]

### 'Sen vikingatid 1050-1200'
late_viking_age_to_early_medieval_period = [
    'v 1051-1071',
    'v 1055-1075',
    'v 1100-t',
    'v/m 1050-1150 eller något senare',
    'v/m 1055-1175',
    'v/m 1065-1185',
    'v/m 1066-1186',
    'v/m 1000-1200',
    'v/m 1000-1200-tal',
    'v/m 1000-1300',
    'v/m ca 1100',
    'v/m 1000-t - 1100-t (arkeologisk datering)',
    'v/m 1000-1100',
    'v/m 1000-1100-t',
    'v/m 1000-1100',
    'v? 1050-1100 (arkeologisk datering)',
    'v/m',
    'v/m 1000-t - 1100-t (arkeologisk datering)',
    'v/m 1000-1100-tal',
    'v/m 1000-1200? (imer 2007)',
    'v? troligen 1000-t',
    'v sen vikingatid el. tidig medeltid', 
    'v eller möjligen m',
    'v? sen vikingatid el. tidig medeltid'
    'v ca 1100',
    'v ca 1100-1130',
    'v efter 1073',
    'v ej yngre än mitten av 1000-t'
]

### 'Vikingatiden 800–1050'
unclear_viking_age = [
    'v efter-jelling(?)',
    'v jelling(?)',
    'v jelling el. efter-jelling',
    'v kristen efter-jelling',
    'v oornerad',
    'v periode 3.2',
    'v ev. sen efterbildning av runsten',
    'v? sen vikingatid el. tidig medeltid',
    'v för-jelling',
    'v helnæs-gørlev',
    'v inte före 915/920',
    'v jelling',
    'v jelling el. kristen efter-jelling',
    'v jelling/efter-jelling'
    ]


In [23]:
# %% Categorizing Dates

# Function to categorize historical periods
def categorize_period(period):

    if period in early_migration_period:
        return 'Folkvandringstid 160–375'
    elif period in late_migration_to_early_vendel:
        return 'Sent folkvandringstid 375–500'
    
    elif period in early_vendel_period:
        return 'Tidig vendeltid 500–600'
    elif period in mid_vendel_period:
        return 'Mellanvendeltid 600–700'
    elif period in late_vendel_to_viking:
        return 'Sent vendeltid 700–800'
    elif period in unclear_vendel_period:
        return 'Vendeltiden 550–800'

    elif period in early_medieval:
        return 'Tidig medeltid 1000–1150'
    elif period in high_medieval:
        return 'Högmedeltid 1150–1300'
    elif period in late_medieval:
        return 'Senmedeltid 1300–1500'
    elif period == 'm' or period in general_medieval:
        return 'Medeltiden 1050–1520'

    elif period in early_modern_period:
        return 'Tidiga nya tiden 1500–1700'
    elif period in eighteenth_century:
        return 'Upplysningstiden 1700–1800'
    elif period in nineteenth_twentieth_century:
        return 'Industrialiseringen 1800–1900'
    elif period in general_sentida:
        return 'Modern tid 1500-nutid'

    elif period in early_viking_age:
            return 'Tidig vikingatiden 700-800'
    elif period in viking_age:
            return 'Vikingatiden 800-1050'
    elif period in late_viking_age_to_early_medieval_period:
            return 'Sen vikingatid 1050-1200'
    elif period in unclear_viking_age:
            return 'Vikingatiden 800–1050'

    elif period in ['v', 'V (?)', 'v (?)', 'v?']:
        return 'Vikingatiden 800–1050'  

    elif period== '?' or period == 'v/m eller sentida?':
        return 'Okänd'
    
    else:
        return period

# Apply the categorization function to the 'datering' column
df['datering'] = df['datering'].apply(func=categorize_period)


In [ ]:
# Sort and display unique dates after cleaning
unique_dates_cleaned = list(df['datering'].unique())
unique_dates_cleaned.sort()
unique_dates_cleaned


In [ ]:
# Dictionary for grouping historical periods
datering_grouping_dict = {
    'Folkvandringstid 160–375': 'Folkvandringstid 160–500',
    'Sent folkvandringstid 375–500': 'Folkvandringstid 160–500',
    'Vendeltiden 550–800': 'Vendeltiden 500–800',
    'Tidig vendeltid 500–600': 'Vendeltiden 500–800',
    'Sent vendeltid 700–800': 'Vendeltiden 500–800',
    'Vikingatiden 800–1050': 'Vikingatiden 800–1050',
    'Vikingatiden 800-1050': 'Vikingatiden 800–1050',
    'Tidig vikingatiden 700-800': 'Vikingatiden 800–1050',
    'Medeltiden 1050–1520': 'Medeltiden 1000–1500',
    'Tidig medeltid 1000–1150': 'Medeltiden 1000–1500',
    'Högmedeltid 1150–1300': 'Medeltiden 1000–1500',
    'Sen vikingatid 1050-1200': 'Medeltiden 1000–1500',
    'Senmedeltid 1300–1500': 'Medeltiden 1000–1500',
    'Tidiga nya tiden 1500–1700': 'Modern tid 1500-nutid',
    'Modern tid 1500-nutid': 'Modern tid 1500-nutid',
    'Upplysningstiden 1700–1800': 'Modern tid 1500-nutid',
    'Industrialiseringen 1800–1900': 'Modern tid 1500-nutid',
    'Okänd': 'missing',
    'missing': 'missing'
}

# Apply grouping to create a new 'grouped_datering' column
df['grouped_datering'] = df['datering'].replace(to_replace= datering_grouping_dict)


In [ ]:
# Sort and display unique grouped dates
unique_dates_cleaned = list(df['grouped_datering'].unique())
unique_dates_cleaned.sort()
unique_dates_cleaned


In [ ]:
# Export the cleaned data to an Excel file

output_file_path = 'cleaned_rundata_24102024.xlsx'
df.to_excel(excel_writer= output_file_path, index=False)